In [ ]:
import pandas as pd
import os
PATH = os.getcwd()
print(PATH)
import glob
import datetime
# to check first rows of files
from csv import reader
# display info options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None  # default='warn'
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
def read_mch_info(file):
    i = 0
    with open(file, 'r') as csv_file:
        csv_reader = reader(csv_file, delimiter=';')
        mch_info = []
        for row in csv_reader:
            # row variable is a list that represents a row in csv
            mch_info.append(row[0])
            i += 1
            if i == 3:
                break

    return(mch_info)

In [ ]:
def concat_files(AllFilesNames):
   # Create an empty list to store the dataframes
    ListDataframe = list()

    # Total file number
    NoFiles = len(AllFilesNames)

    # For each file in the specified directory import the data and add it in the list
    for Filename in AllFilesNames:
        ListDataframe.append(pd.read_csv(Filename, sep=';', skiprows=3, decimal=',', encoding='unicode_escape'))

    # Concatenate the files in the list in unique dataframe
    DF_Data = pd.concat(ListDataframe, axis=0, ignore_index=True)

    # DateTime
    DF_Data['DateTime'] = pd.to_datetime(DF_Data['DateTime'], format="%Y-%m-%d %H:%M:%S")
    # ordering ascending
    DF_Data = DF_Data.sort_values(by='DateTime', ascending=True).reset_index(drop=True)

    #Remove duplicates
    DF_Data.drop_duplicates(keep=False, inplace=True)

    return(DF_Data)

In [ ]:
# Folder name / TODO: GET FROM USER
foldername = "VisuaLite/FCM_B_C/FCM_C/LogExport202107061025/"

In [ ]:
# Define files path
Name = "S?*.csv"
# Read all the files name in the directory
DataFiles = glob.glob(foldername + Name)

# Define files path
Name = "A?*.csv"
# Read all the files name in the directory
AlarmFiles = glob.glob(foldername + Name)

# Define files path
Name = "E?*.csv"
# Read all the files name in the directory
EventFiles = glob.glob(foldername + Name)

print(DataFiles)
print(AlarmFiles)
print(EventFiles)

In [ ]:
def check_files (files):
    # Check Machine information
    mch_info_check = read_mch_info(files[0])

    for file in (files):
        mch_info = read_mch_info(file)
        if mch_info != mch_info_check:
            print('File ' + file + ' does not correspond to the same machine')
            print(mch_info_check)
            return 0, mch_info_check
    
    print('All files correspond to the same machine')
    print(mch_info_check)
    return 1, mch_info_check

In [ ]:
flag_files, mch_info = check_files(DataFiles + AlarmFiles + EventFiles)
if flag_files:
    print('all good')

In [ ]:
LogsStandard = concat_files(DataFiles)
print(LogsStandard.columns)
print(LogsStandard.shape)

LogsAlarms = concat_files(AlarmFiles)
print(LogsAlarms.columns)
print(LogsAlarms.shape)

LogsEvents = concat_files(EventFiles)
print(LogsEvents.columns)
print(LogsEvents.shape)


In [ ]:
# Format LogsStandard
LogsStandard = LogsStandard[['DateTime','GpsPos', 'ChangeOverInProgress','F60InAutoMode','CV1_Position', 'CV2_Position',
                  'CV3_Position', 'CV4_Position', 'CV5_Position', 'SupplyCurrentPump',
       'CircCurrentPump', 'CurrentControl', 'CurrentFilter',
       'PT1', 'PT2', 'TT1', 'TT2', 'TargetTemperature',
       'TemperatureLowLimit', 'TemperatureHighLimit', 'VT', 'TargetViscosity',
       'ViscosityLowLimit', 'ViscosityHighLimit', 'Density', 'FM1_MassFlow',
       'FM1_Density', 'FM1_Temperature', 'FM2_MassFlow', 'FM2_Density',
       'FM2_Temperature', 'FM3_MassFlow', 'FM3_Density', 'FM3_Temperature',
       'FM4_MassFlow', 'FM4_Density', 'FM4_Temperature', 'FT_VolumeFlow',
       'FT_MassFlow', 'FT_Density', 'FT_Temperature', 'SO2', 'CO2', 'SC','DPT_AI']]

# Columns as float
LogsStandard.iloc[:,13:] = LogsStandard.iloc[:,13:].astype(float)

LogsStandard[['CV1_Label','CV2_Label','CV3_Label','CV4_Label','CV5_Label']] = LogsStandard[['CV1_Position','CV2_Position','CV3_Position','CV4_Position','CV5_Position']]

LogsStandard[['CV1_Label']] = LogsStandard[['CV1_Label']] .replace({ 
    0: "Both LS activated",
    1: "Fuel 1 Position",
    2: "No LS activated",
    3: "Other fuel Position"
})

LogsStandard[['CV2_Label']] = LogsStandard[['CV2_Label']] .replace({ 
    0: "Both LS activated",
    1: "Fuel 2 Position",
    2: "No LS activated",
    3: "Other Fuel Position"
})

LogsStandard[['CV3_Label']] = LogsStandard[['CV3_Label']] .replace({ 
    0: "Both LS activated",
    1: "Fuel 3 Position",
    2: "No LS activated",
    3: "Fuel 4 Position"
})

LogsStandard[['CV4_Label']] = LogsStandard[['CV4_Label']] .replace({ 
    0: "Both LS activated",
    1: "Heater Position",
    2: "No LS activated",
    3: "Cooler Position"
})

LogsStandard[['CV5_Label']] = LogsStandard[['CV5_Label']] .replace({ 
    0: "Both LS activated",
    1: "Cooler Position",
    2: "No LS activated",
    3: "Bypass Position"
})

LogsStandard.dtypes

In [ ]:
LogsAlarms = LogsAlarms[['DateTime', 'AlarmNumber']]

LogsAlarms['Label'] = LogsAlarms['AlarmNumber'] 

LogsAlarms[['Label']] = LogsAlarms[['Label']] .replace({ 
    0: "PLC battery low / Not present",
    1: "Dynamic I/O Configuration Error",
    2: "Dynamic I/O Configuration In Progress",
    3: "Dynamic I/O Configuration Done",
    4: "Invalid GPS Sig-nal ",
    5: "I/O Module Error",
    102: "Emergency stop",
    103: "Power failure",
    104: "Instrument Mod-bus Communi-cation Error",
    105: "UPS Battery Low",
    106: "Local HMI com-munication lost",
    107: "Remote HMI communication lost",
    200: "Changeover Valve 1 Alarm",
    201: "Changeover Valve 2 Alarm",
    202: "Changeover Valve 3 Alarm",
    203: "Changeover Finished",
    300: "SCT sensor fault, signal missing",
    301: "Fuel consump-tion too low for blending",
    400: "Supply Pump 1 failure",
    401: "Supply Pump 2 failure",
    402: "Supply pump 1 Not Available",
    403: "Supply pump 2 Not Available",
    404: "PT1 limit high",
    405: "PT1 limit low",
    406: "PT1 sensor fault, signal missing",
    408: "Supply Pump switch over done",
    409: "Automatic switch of Supply Pumps in a short time",
    410: "Standby Supply Pump run time reset to zero",
    411: "Supply Pump switch time elapsed",
    412: "Supply Pump switching failed",
    413: "Standby Supply Pump not avail-able Please re-store it",
    414: "Warning/Alarm on VFD 1",
    415: "Warning/Alarm on VFD 2",
    500: "Circulation Pump 1 failure",
    501: "Circulation Pump 2 failure",
    502: "Circulation pump 1 Not Available",
    503: "Circulation pump 2 Not Available",
    504: "PT2 limit high",
    505: "PT2 limit low",
    506: "PT2 sensor fault, signal missing",
    508: "Circulation Pump switch-over done",
    509: "Automatic switch of Circulation Pumps in a short time",
    510: "Standby Circulation Pump run time reset to zero",
    511: "Circulation Pump switch time elapsed",
    512: "Circulation Pump switching failed ",
    513: "Standby Circula-tion Pump not available. Please restore it.",
    600: "Automatic Filter Differential Pressure high",
    601: "Automatic Filter Failure",
    602: "DPT Filter Fault",
    800: "Mixing tank low level",
    900: "Electric Heater 1-Fault",
    901: "Electric Heater 2-Fault",
    902: "Fuel Temperature High",
    903: "Fuel Temperature Low",
    905: "Chiller Fault",
    906: "Viscosity not reached during Changeover.",
    907: "TT1 sensor fault, signal missing",
    908: "TT2 sensor fault, signal missing",
    911: "CV4 Alarm",
    912: "CV5 Alarm",
    913: "Switched to Vis-cosity control",
    914: "Switched to Temperature control",
    915: "Temperature control out of order, will be switched off within 5 minutes",
    916: "Temperature control switched off",
    917: "Very high temperature reached",
    1000: "Viscosity Sensor Fault",
    1002: "Density Sensor Fault",
    1004: "Fuel viscosity High limit alarm",
    1005: "Fuel viscosity Low limit alarm"
})

LogsAlarms.dtypes

In [ ]:
LogsEvents = LogsEvents[['DateTime', 'GpsPos', 'EventNumber', 'Data']]
LogsEvents['Label'] = LogsEvents['EventNumber'] 

LogsEvents[['Label']] = LogsEvents[['Label']] .replace({0 : 'FCM Started',
                                                        1 : 'FCM Stopped',
                                                        2 : 'Changeover initiated',
                                                        3 : 'Changeover Finsihed',
                                                        4 : 'Any device in Manual Mode',
                                                        5 : 'All device in Auto Mode',
                                                        6 : 'P401 SPump value changed',
                                                        7 : 'P501 CPump value changed',
                                                        8 : 'P903 Heater In Use value changed',
                                                        30 : 'Auto Mode Selected',
                                                        31 : 'Manual Mode Selected'})

LogsEvents.loc[(LogsEvents['EventNumber'] == 6) |
               (LogsEvents['EventNumber'] == 7) | 
               (LogsEvents['EventNumber'] == 8), 'Label'] = LogsEvents['Label'].astype(str) + ": " + LogsEvents['Data'].astype(str)


LogsEvents.head()

# Change Overs

In [ ]:
# Identify when Change Over Started and Finished
# ChangeOverInProgress = O : no change
# ChangeOverInProgress = 1 : started
# ChangeOverInProgress = -1 : finished

# Create column with value change
LogsStandard['ChangeoverCMDchange'] = LogsStandard['ChangeOverInProgress'].diff()

# memorize starting DateTimes
COstart = LogsStandard[LogsStandard['ChangeoverCMDchange'] == 1]['DateTime'].tolist()

# memorize finishing DateTimes
COfinish = LogsStandard[(LogsStandard['ChangeoverCMDchange'] == -1)]['DateTime'].tolist()

COs = []
for i in range(len(COfinish)):
    duration = COfinish[i]-COstart[i]
    if duration > datetime.timedelta(minutes = 1):
        COs.append({'Start': COstart[i], 'Finish': COfinish[i], 'Duration': duration})

print('In the logs imported there are ' + str(len(COs)) + ' changeovers')
for CO in COs:
    print('- From ' + str(CO['Start']) + ' to ' + str(CO['Finish']) + '. Duration: ' + str(CO['Duration'])) 


In [ ]:
def ChangeOverToDF(CO):
    delta = datetime.timedelta(minutes = 20)
    df = LogsStandard[(LogsStandard['DateTime'] >= CO['Start']-delta) & (LogsStandard['DateTime'] <= CO['Finish']+delta)]
    return(df)

# PLOTS

In [ ]:
greens = ['rgba(0, 128, 0, 0.1)', #green opacity 10%
          'rgba(196, 180, 84, 1)', #vegas gold
          'rgba(64, 224, 208, 1)', #turquoise
          'rgba(69, 75, 27, 1)', #army green
          'rgba(147, 197, 114, 1)', #pistachio
          'rgba(8, 143, 143, 1)', #citrine
        ]

blues = ['rgba(0, 71, 171, 1)', #cobalt blue
        'rgba(70, 130, 180, 1)' #steel blue
        ]

reds = ['rgba(255, 87, 51, 1)', #cherry
        'rgba(250, 160, 160, 1)', #pastel red
        ]

purples = ['rgba(191, 64, 191, 1)', #bright purple
           'rgba(93, 63, 211, 1)', #iris
           'rgba(127, 0, 255, 1)', #violet
           'rgba(149, 53, 83, 1)', #red purple
           'rgba(218, 112, 214, 1)' #mulberry
           ]

comp1 = ['rgba(29, 157, 228, 1)', #blue*
           'rgba(228, 100, 29, 1)', #orange*
           'rgba(63, 166, 121, 1)', #green*
           ]

ALcolors = ['rgba(17, 56, 127, 1)', #AL blue
            'rgba(0, 0, 0, 1)', #AL white
            'rgba(220, 146, 118, 1)', #AL earth
            'rgba(254, 205, 96, 1)', #AL sun
            'rgba(147, 199, 198, 1)', #AL water
            'rgba(0, 127, 200, 1)', #AL innovation
            ]

In [ ]:
def Plot_ChangeOver_simple(df):
    fig = go.Figure()

    # --------------------------------------------------- Valves
    greens = ['rgba(0, 128, 0, 0.1)', #green opacity 10%
            'rgba(196, 180, 84, 1)', #vegas gold
            'rgba(64, 224, 208, 1)', #turquoise
            'rgba(69, 75, 27, 1)', #army green
            'rgba(147, 197, 114, 1)', #pistachio
            'rgba(8, 143, 143, 1)', #citrine
            ]

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['ChangeOverInProgress'].astype(bool),
        name="Change Over InProgress",
        fill='tozeroy', mode='none', fillcolor = greens[0],    
        yaxis="y4"
    ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV1_Position'],
        name="CV1 Position",
        line=dict(color = greens[1]), 
        legendgroup="Valves",legendgrouptitle_text="Valves",
        visible='legendonly',
        yaxis="y3"
    ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV2_Position'],
        name="CV2 Position",
        line=dict(color = greens[2]),
        legendgroup="Valves",legendgrouptitle_text="Valves",
        visible='legendonly',
        yaxis="y3"
    ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV3_Position'],
        name="CV3 Position",
        line=dict(color = greens[3]),    
        legendgroup="Valves",legendgrouptitle_text="Valves",
        visible='legendonly',
        yaxis="y3"
    ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV4_Position'],
        name="CV4 Position",
        line=dict(color = greens[4]), 
        legendgroup="Valves",legendgrouptitle_text="Valves",
        visible='legendonly',
        yaxis="y3"
    ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV5_Position'],
        name="CV5 Position",
        line=dict(color = greens[5]),
        legendgroup="Valves",legendgrouptitle_text="Valves",
        visible='legendonly',
        yaxis="y3"
    ))

    # ---------------------------------------------------Temperature
    reds = ['rgba(255, 87, 51, 1)', #cherry
            'rgba(250, 160, 160, 1)' #pastel red
            ]
    if (df['TT1'] != 0).all():
        fig.add_trace(go.Scatter(
            x=df['DateTime'],
            y=df['TT1'],
            name='TT1 - Act Temp',
            line=dict(color = reds[0]),
            legendgroup="Temperature",legendgrouptitle_text="Temperature",
        ))

    if (df['TT2'] != 0).all():
        fig.add_trace(go.Scatter(
            x=df['DateTime'],
            y=df['TT2'],
            name='TT2 - Act Temp',
            line=dict(color = reds[0]),
            legendgroup="Temperature",legendgrouptitle_text="Temperature",
        ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TargetTemperature'],
        name='Target Temperature',
        line=dict(color = reds[1]),
        legendgroup="Temperature",legendgrouptitle_text="Temperature",
    ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TemperatureLowLimit'],
        name='Temp Low Limit',
        line=dict(color = reds[1], dash='dot'),
        legendgroup="Temperature",legendgrouptitle_text="Temperature",
    ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TemperatureHighLimit'],
        name='Temperature High Limit',
        line=dict(color = reds[1], dash='dot'),
        legendgroup="Temperature",legendgrouptitle_text="Temperature",
    ))

    # --------------------------------------------------- Viscosity
    blues = ['rgba(0, 71, 171, 1)', #cobalt blue
            'rgba(70, 130, 180, 1)' #steel blue
            ]

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['VT'],
        name='Viscosity',
        line=dict(color = blues[0]),
        legendgroup="Viscosity",legendgrouptitle_text="Viscosity",
        yaxis="y2"))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TargetViscosity'],
        name='Target Viscosity',
        line=dict(color = blues[1]),
        legendgroup="Viscosity",legendgrouptitle_text="Viscosity",
        yaxis="y2"))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['ViscosityLowLimit'],
        name='Viscosity Low Limit',
        line=dict(color = blues[1], dash='dot'),
        legendgroup="Viscosity",legendgrouptitle_text="Viscosity",
        yaxis="y2"))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['ViscosityHighLimit'],
        name='Viscosity High Limit',
        line=dict(color = blues[1], dash='dot'),
        legendgroup="Viscosity",legendgrouptitle_text="Viscosity",
        yaxis="y2"))

    # Create axis objects
    fig.update_layout(
        xaxis=dict(domain=[0.1, 0.9]), #compress x axis 10% left an right

        yaxis=dict(
            title="Temperature"),
        yaxis2=dict(
            title="Viscosity",
            anchor="free",
            overlaying="y",
            side="left",
            position=0
        ),
        yaxis3=dict(
            title="CV Position",
            anchor="x",
            overlaying="y",
            side="right"
        ),
        yaxis4=dict(
            title="Change Over",
            anchor="free",
            overlaying="y",
            side="right",
            position=1
        ),
        legend=dict(orientation="v", x = 1.1)
    )

    # Update layout properties
    fig.update_layout(
        title_text=("ChangeOver and CVs"),
        legend=dict(groupclick="toggleitem") #avoid grouping all traces
    )

    return (fig)

In [ ]:
def Plot_ChangeOver(df, mch_info):
    mindate = df['DateTime'].min()
    maxdate = df['DateTime'].max()
    print(mindate,maxdate)

    alm = pd.DataFrame()
    alm = LogsAlarms[(LogsAlarms['DateTime'] > mindate) & (LogsAlarms['DateTime'] <= maxdate)]

    eve = pd.DataFrame()
    eve = LogsEvents[(LogsEvents['DateTime'] > mindate) & (LogsEvents['DateTime'] <= maxdate)]

    fig = make_subplots(
        rows=8, cols=1,
        specs=[[{"rowspan": 4, "colspan": 1, "secondary_y": True}], # 4 Rows for Visc and Temp Trends
            [None],
            [None],
            [None],
            [{"secondary_y": True}],  # Valves position & ChangeOverON
            [{"secondary_y": True}],  # FT & Density
            [{"secondary_y": True}],  # PT & Alarms
            [{"secondary_y": True}]], # Events & Alarms
            shared_xaxes=True,
        print_grid=True)

    # --------------------------------------------------------------------------------------------- R1-4 C1
    plot_row= 1
    plot_col= 1

    fig.update_yaxes(title_text="Temperature", 
                    title_font_color= reds[0],
                    color= reds[0],row=plot_row, secondary_y=False)
    fig.update_yaxes(title_text="Viscosity", 
                    title_font_color= blues[0],
                    color= blues[0], row=plot_row, secondary_y=True)

    # ---------------------------------------------------Temperature
    if (df['TT1'] != 0).all():
        fig.add_trace(go.Scatter(
            x=df['DateTime'],
            y=df['TT1'],
            name='TT1 - Act Temp',
            line=dict(color = reds[0]),
            legendgroup="Temperature",legendgrouptitle_text="Temperature",
        ), row=plot_row, col=plot_col, secondary_y=False)

    if (df['TT2'] != 0).all():
        fig.add_trace(go.Scatter(
            x=df['DateTime'],
            y=df['TT2'],
            name='TT2 - Act Temp',
            line=dict(color = reds[0]),
            legendgroup="Temperature",legendgrouptitle_text="Temperature",
        ), row=plot_row, col=plot_col, secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TargetTemperature'],
        name='Target Temperature',
        line=dict(color = reds[1]),line_shape='hv',
        legendgroup="Temperature",legendgrouptitle_text="Temperature",
    ), row=plot_row, col=plot_col, secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TemperatureLowLimit'],
        name='Temp Low Limit',
        line=dict(color = reds[1], dash='dot'),line_shape='hv',
        legendgroup="Temperature",legendgrouptitle_text="Temperature",
    ), row=plot_row, col=plot_col, secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TemperatureHighLimit'],
        name='Temperature High Limit',
        line=dict(color = reds[1], dash='dot'),line_shape='hv',
        legendgroup="Temperature",legendgrouptitle_text="Temperature",
    ), row=plot_row, col=plot_col, secondary_y=False)

    # --------------------------------------------------- Viscosity

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['VT'],
        name='Viscosity',
        line=dict(color = blues[0]),
        legendgroup="Viscosity",legendgrouptitle_text="Viscosity"),
        row=plot_row, col=plot_col, secondary_y=True)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TargetViscosity'],
        name='Target Viscosity',
        line=dict(color = blues[1]),line_shape='hv',
        legendgroup="Viscosity",legendgrouptitle_text="Viscosity"),
        row=plot_row, col=plot_col, secondary_y=True)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['ViscosityLowLimit'],
        name='Viscosity Low Limit',
        line=dict(color = blues[1], dash='dot'),line_shape='hv',
        legendgroup="Viscosity",legendgrouptitle_text="Viscosity"),
        row=plot_row, col=plot_col, secondary_y=True)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['ViscosityHighLimit'],
        name='Viscosity High Limit',
        line=dict(color = blues[1], dash='dot'),line_shape='hv',
        legendgroup="Viscosity",legendgrouptitle_text="Viscosity"),
        row=plot_row, col=plot_col, secondary_y=True)

    # --------------------------------------------------------------------------------------------- R5 C1
    plot_row= 5
    plot_col= 1

    fig.update_yaxes(visible = False,
                    title_text="True/False",
                    row=plot_row, secondary_y=True)
    fig.update_yaxes(title_text="Valve Position",
                    row=plot_row, secondary_y=False)
    # --------------------------------------------------- Bools
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['ChangeOverInProgress'].astype(bool),
        name="Change Over InProgress",
        fill='tozeroy', mode='none', fillcolor = greens[0],    
        line_shape='hv'
    ), row=plot_row, col=plot_col, secondary_y=True)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV1_Position'],
        name="CV1 Position",
        line=dict(color = ALcolors[0]), 
        legendgroup="Valves",legendgrouptitle_text="Valves", line_shape='hv',
        hovertext=df['CV1_Label']), row=plot_row, col=plot_col, secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV2_Position'],
        name="CV2 Position",
        line=dict(color = ALcolors[2]),
        legendgroup="Valves",legendgrouptitle_text="Valves", line_shape='hv',
        hovertext=df['CV2_Label']), row=plot_row, col=plot_col, secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV3_Position'],
        name="CV3 Position",
        line=dict(color = ALcolors[3]),    
        legendgroup="Valves",legendgrouptitle_text="Valves", line_shape='hv',
        hovertext=df['CV3_Label']), row=plot_row, col=plot_col, secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV4_Position'],
        name="CV4 Position",
        line=dict(color = ALcolors[4]), 
        legendgroup="Valves",legendgrouptitle_text="Valves", line_shape='hv',
        hovertext=df['CV4_Label']), row=plot_row, col=plot_col, secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV5_Position'],
        name="CV5 Position",
        line=dict(color = ALcolors[5]),
        legendgroup="Valves",legendgrouptitle_text="Valves", line_shape='hv',
        hovertext=df['CV5_Label']), row=plot_row, col=plot_col, secondary_y=False)

    # --------------------------------------------------------------------------------------------- R6 C1
    plot_row= 6
    plot_col= 1

    fig.update_yaxes(title_text="Flow",
                    row=plot_row, secondary_y=False)
    fig.update_yaxes(title_text="Density",
                    row=plot_row, secondary_y=True)
    # --------------------------------------------------- Flow Meter
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['FT_MassFlow'],
        name='FT_MassFlow',
        line=dict(color = purples[0]),
        legendgroup="FT",legendgrouptitle_text="Flow"),
        row=plot_row, col=plot_col, secondary_y=False)
        
    if (df['FM1_MassFlow'] != 0).all():
        fig.add_trace(go.Scatter(
            x=df['DateTime'],
            y=df['FM1_MassFlow'],
            name='FM1 MassFlow',
            line=dict(color = purples[1]),
            legendgroup="FT",legendgrouptitle_text="Flow"),
            row=plot_row, col=plot_col, secondary_y=False)

    if (df['FM2_MassFlow'] != 0).all():
        fig.add_trace(go.Scatter(
            x=df['DateTime'],
            y=df['FM2_MassFlow'],
            name='FM2 MassFlow',
            line=dict(color = purples[2]),
            legendgroup="FT",legendgrouptitle_text="Flow"),
            row=plot_row, col=plot_col, secondary_y=False)
        
    if (df['FM3_MassFlow'] != 0).all():
        fig.add_trace(go.Scatter(
            x=df['DateTime'],
            y=df['FM3_MassFlow'],
            name='FM3 MassFlow',
            line=dict(color = purples[3]),
            legendgroup="FT",legendgrouptitle_text="Flow"),
            row=plot_row, col=plot_col, secondary_y=False)

    if (df['FM4_MassFlow'] != 0).all():
        fig.add_trace(go.Scatter(
            x=df['DateTime'],
            y=df['FM4_MassFlow'],
            name='FM4 MassFlow',
            line=dict(color = purples[4]),
            legendgroup="FT",legendgrouptitle_text="Flow"),
            row=plot_row, col=plot_col, secondary_y=False)

    # --------------------------------------------------- Density
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['Density'],
        name='Density',
        line=dict(color = comp1[2]),
        legendgroup="Density",legendgrouptitle_text="Density"),
        row=plot_row, col=plot_col, secondary_y=True)
        
    # --------------------------------------------------------------------------------------------- R7 C1
    plot_row= 7
    plot_col= 1

    fig.update_yaxes(title_text="Pressure",
                    row=plot_row, secondary_y=False)
    fig.update_yaxes(title_text="Alarms",
                    row=plot_row, secondary_y=True)
    # --------------------------------------------------- Pressures
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['PT1'],
        name='PT1',
        line=dict(color = comp1[0]),
        legendgroup="PT",legendgrouptitle_text="Pressure"),
        row=plot_row, col=plot_col, secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['PT2'],
        name='PT2',
        line=dict(color = comp1[1]),
        legendgroup="PT",legendgrouptitle_text="Pressure"),
        row=plot_row, col=plot_col, secondary_y=False)

    # --------------------------------------------------- Alarms
    p_alm = alm[(alm['AlarmNumber'] > 399) & (alm['AlarmNumber'] <= 499)] # Pumps Alarms A4xx
    if not p_alm.empty:
        fig.add_trace(go.Scatter(x=p_alm['DateTime'], y=p_alm['AlarmNumber'].astype(str),
                                name='Pressure Alarms',
                                mode='markers', marker_symbol='x',
                                marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                                marker_line_width=1, marker_size=8,
                                legendgroup="Alarms",legendgrouptitle_text="SPump Alarms"),
                        row=plot_row, col=plot_col,secondary_y=True)   
        
    p_alm = alm[(alm['AlarmNumber'] > 499) & (alm['AlarmNumber'] <= 599)] # Pumps Alarms A5xx
    if not p_alm.empty:
        fig.add_trace(go.Scatter(x=p_alm['DateTime'], y=p_alm['AlarmNumber'].astype(str),
                                name='Pressure Alarms',
                                mode='markers', marker_symbol='x',
                                marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                                marker_line_width=1, marker_size=8,
                                legendgroup="Alarms",legendgrouptitle_text="CPump Alarms"),
                        row=plot_row, col=plot_col,secondary_y=True)    

    # --------------------------------------------------------------------------------------------- R8 C1
    plot_row= 8
    plot_col= 1

    fig.update_yaxes(title_text="Events",
                    row=plot_row, secondary_y=False)
    fig.update_yaxes(title_text="Alarms",
                    row=plot_row, secondary_y=True)
    # --------------------------------------------------- Events
    if not eve.empty:
        fig.add_trace(go.Scatter(x=eve['DateTime'], y=eve['EventNumber'].astype(int),
                                name='Event Number',
                                mode='markers',marker_symbol='star',
                                marker_line_color=ALcolors[5], marker_color=ALcolors[5],
                                marker_line_width=1, marker_size=8,
                                legendgroup="Events",legendgrouptitle_text="Events",
                                hovertext=eve['Label']),
                        row=plot_row, col=plot_col,secondary_y=False)
        

    # --------------------------------------------------- Alarms
    p_alm = alm[(alm['AlarmNumber'] < 199)] # System Alarms A0xx to A1xx
    if not p_alm.empty:
        fig.add_trace(go.Scatter(x=p_alm['DateTime'], y=p_alm['AlarmNumber'].astype(str),
                                name='System Alarms',
                                mode='markers', marker_symbol='x',
                                marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                                marker_line_width=1, marker_size=8,
                                legendgroup="Alarms",legendgrouptitle_text="Alarms",
                                hovertext=p_alm['Label']),
                        row=plot_row, col=plot_col,secondary_y=True)     
        

    p_alm = alm[(alm['AlarmNumber'] > 199) & (alm['AlarmNumber'] <= 299)] # ChangeOver Alarms A2xx
    if not p_alm.empty:
        fig.add_trace(go.Scatter(x=p_alm['DateTime'], y=p_alm['AlarmNumber'].astype(str),
                                name='ChangeOver Alarms',
                                mode='markers', marker_symbol='x',
                                marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                                marker_line_width=1, marker_size=8,
                                legendgroup="Alarms",legendgrouptitle_text="Alarms",
                                hovertext=p_alm['Label']),
                        row=plot_row, col=plot_col,secondary_y=True)     
        
    p_alm = alm[(alm['AlarmNumber'] > 299) & (alm['AlarmNumber'] <= 399)] # Blending Alarms A3xx
    if not p_alm.empty:
        fig.add_trace(go.Scatter(x=p_alm['DateTime'], y=p_alm['AlarmNumber'].astype(str),
                                name='Blending Alarms',
                                mode='markers', marker_symbol='x',
                                marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                                marker_line_width=1, marker_size=8,
                                legendgroup="Alarms",legendgrouptitle_text="Alarms",
                                hovertext=p_alm['Label']),
                        row=plot_row, col=plot_col,secondary_y=True) 
        
    p_alm = alm[(alm['AlarmNumber'] > 599) & (alm['AlarmNumber'] <= 699)] # Filter Alarms A6xx
    if not p_alm.empty:
        fig.add_trace(go.Scatter(x=p_alm['DateTime'], y=p_alm['AlarmNumber'].astype(str),
                                name='Filter Alarms',
                                mode='markers', marker_symbol='x',
                                marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                                marker_line_width=1, marker_size=8,
                                legendgroup="Alarms",legendgrouptitle_text="Alarms",
                                hovertext=p_alm['Label']),
                        row=plot_row, col=plot_col,secondary_y=True) 
        
    p_alm = alm[(alm['AlarmNumber'] > 799) & (alm['AlarmNumber'] <= 899)] # Mixing Tank Alarms A8xx
    if not p_alm.empty:
        fig.add_trace(go.Scatter(x=p_alm['DateTime'], y=p_alm['AlarmNumber'].astype(str),
                                name='Mixing Tank',
                                mode='markers', marker_symbol='x',
                                marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                                marker_line_width=1, marker_size=8,
                                legendgroup="Alarms",legendgrouptitle_text="Alarms",
                                hovertext=p_alm['Label']),
                        row=plot_row, col=plot_col,secondary_y=True) 
        
    p_alm = alm[(alm['AlarmNumber'] > 899) & (alm['AlarmNumber'] <= 999)] # TempControl Alarms A9xx
    if not p_alm.empty:
        fig.add_trace(go.Scatter(x=p_alm['DateTime'], y=p_alm['AlarmNumber'].astype(str),
                                name='TempControl Alarms',
                                mode='markers', marker_symbol='x',
                                marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                                marker_line_width=1, marker_size=8,
                                legendgroup="Alarms",legendgrouptitle_text="Alarms",
                                hovertext=p_alm['Label']),
                        row=plot_row, col=plot_col,secondary_y=True) 
        
    p_alm = alm[(alm['AlarmNumber'] > 999) & (alm['AlarmNumber'] <= 1099)] # ViscMeas Alarms A9xx
    if not p_alm.empty:
        fig.add_trace(go.Scatter(x=p_alm['DateTime'], y=p_alm['AlarmNumber'].astype(str),
                                name='ViscMeas Alarms',
                                mode='markers', marker_symbol='x',
                                marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                                marker_line_width=1, marker_size=8,
                                legendgroup="Alarms",legendgrouptitle_text="Alarms",
                                hovertext=p_alm['Label']),
                        row=plot_row, col=plot_col,secondary_y=True) 

    # Update layout properties
    fig.update_layout(hovermode="x unified", hoverlabel=dict(bgcolor='rgba(255,255,255,0.75)', namelength = -1, font=dict(color='black')),  
        legend=dict(groupclick="toggleitem"), #avoid grouping all traces #orientation="v", x = 1.1, 
        title_text=(mch_info[0] + " | " + mch_info[1] + " | " + mch_info[2]) , title_x=0.5
    )


    # Add image
    from PIL import Image
    alLogo = Image.open(PATH + "\VisuaLite\\resources\ALlogo.png")
    fig.add_layout_image(
        dict(
            source=alLogo,
            xref="paper", yref="paper",
            x=0, y=1.025,
            sizex=0.14, sizey=0.14,
            xanchor="left", yanchor="bottom"
        )
    )

    return(fig)


In [ ]:
df = ChangeOverToDF(COs[1])

fig = Plot_ChangeOver(df, mch_info)
fig.write_html("ChangeOver_test4ROWS.html", config={'displaylogo': False})
#fig.update_layout(autosize=False, width=1000, height=800)
fig.show(config={'displaylogo': False})

fig2 = Plot_ChangeOver_simple(df)
fig2.write_html("ChangeOver_test1ROW.html", config={'displaylogo': False})
#fig2.update_layout(autosize=False, width=1000, height=800)
#fig2.show(config={'displaylogo': False})


# STAMPA REPORT

Tutti Par ChangeOver, Fuel, Temp, Visc

# VisuaLite

In [ ]:
import tkinter as tk
import customtkinter as ctk
import tkinter.filedialog as fd
from PIL import Image
import os

PATH = os.getcwd()
print(PATH)

ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("dark-blue")

class ScrollableCheckBoxFrame(ctk.CTkScrollableFrame):
    def __init__(self, master, item_list, command=None, **kwargs):
        super().__init__(master, **kwargs)

        self.command = command
        self.checkbox_list = []
        for i, item in enumerate(item_list):
            self.add_item(item)

    def add_item(self, item):
        checkbox = ctk.CTkCheckBox(self, text=item)
        if self.command is not None:
            checkbox.configure(command=self.command)
        checkbox.grid(row=len(self.checkbox_list), column=0, padx=5, pady=10, sticky="w")
        checkbox.select()
        self.checkbox_list.append(checkbox)

    def get_checked_items(self):
        return [checkbox.cget("text") for checkbox in self.checkbox_list if checkbox.get() == 1]
    
class BreadcrumbFrame(ctk.CTkFrame):
    def __init__(self, master, **kwargs):
        super().__init__(master, **kwargs)

        # Load the images
        image1_path = os.path.join(PATH, "number-1.png")
        self.step1_img = Image.open(image1_path).resize((30, 30))
        self.step1_img_tk = ctk.CTkImage(self.step1_img)

        image2_path = os.path.join(PATH, "number-2.png")
        self.step2_img = Image.open(image2_path).resize((30, 30))
        self.step2_img_tk = ctk.CTkImage(self.step2_img)

        image3_path = os.path.join(PATH, "number-3.png")
        self.step3_img = Image.open(image3_path).resize((30, 30))
        self.step3_img_tk = ctk.CTkImage(self.step3_img)

        # STEP 1
        self.step1 = ctk.CTkFrame(self)
        self.step1.grid(row=0, column=0, padx=5, pady=5, sticky="nsew")
        self.step1_label = ctk.CTkLabel(self.step1, text="   Import Files", image=self.step1_img_tk,
                                         compound="left", font=ctk.CTkFont(size=15, weight="bold"))
        self.step1_label.grid(row=0, column=0, padx=20, pady=10)

        # STEP 2
        self.step2 = ctk.CTkFrame(self)
        self.step2.grid(row=0, column=1, padx=5, pady=5, sticky="nsew")
        self.step2_label = ctk.CTkLabel(self.step2, text="   Analysis Type", image=self.step2_img_tk,
                                         compound="left", font=ctk.CTkFont(size=15, weight="bold"))
        self.step2_label.grid(row=0, column=0, padx=20, pady=10)

        # STEP 3
        self.step3 = ctk.CTkFrame(self)
        self.step3.grid(row=0, column=2, padx=5, pady=5, sticky="nsew")
        self.step3_label = ctk.CTkLabel(self.step3, text="   Results", image=self.step3_img_tk,
                                         compound="left", font=ctk.CTkFont(size=15, weight="bold"))
        self.step3_label.grid(row=0, column=0, padx=20, pady=10)

        #configure grid
        self.grid_rowconfigure(0, weight=1)
        self.grid_columnconfigure(0, weight=1)

class App(ctk.CTk):

    frames = {} #dictionary containing frames
    current = None #class ctkFrame of current frame selected
    dirname = None #folder with logs

    def left_side_widgets(self, parent):
        # create sidebar logo
        self.logo_label = ctk.CTkLabel(parent, text="VisuaLite", font=ctk.CTkFont(size=22, weight="bold"))
        self.logo_label.grid(row=0, column=0, padx=20, pady=(20, 10))
        # create info label
        self.info_label = ctk.CTkLabel(parent, text="FCM One | 1.5", font=ctk.CTkFont(size=15))
        self.info_label.grid(row=1, column=0, padx=20, pady=(20, 10))
        # make middle empty row have the priority
        parent.grid_rowconfigure(2, weight=1)
        # create app controls of appearance and scaling
        self.appearance_mode_label = ctk.CTkLabel(parent, text="Appearance Mode:", anchor="w")
        self.appearance_mode_label.grid(row=3, column=0, padx=20, pady=(10, 0))
        self.appearance_mode_optionemenu = ctk.CTkOptionMenu(parent, values=["Light", "Dark"],command=self.change_appearance_mode_event)
        self.appearance_mode_optionemenu.grid(row=4, column=0, padx=20, pady=(10, 10))
        self.appearance_mode_optionemenu.set("Dark")
        self.scaling_label = ctk.CTkLabel(parent, text="UI Scaling:", anchor="w")
        self.scaling_label.grid(row=5, column=0, padx=20, pady=(10, 0))
        self.scaling_optionemenu = ctk.CTkOptionMenu(parent, values=["80%", "90%", "100%", "110%", "120%"],command=self.change_scaling_event)
        self.scaling_optionemenu.grid(row=6, column=0, padx=20, pady=(10, 20))
        self.scaling_optionemenu.set("100%")

    def current_step(self, parent, frame_id):
        # Add a frame on top of the right panel with blue background
        App.frames[frame_id] = ctk.CTkFrame(parent)
        App.frames[frame_id].grid(row=1, column=0,  padx=5, pady= 5, sticky="nsew")

        # create Title and command row
        self.current_step = ctk.CTkLabel(App.frames[frame_id], fg_color="transparent", text="Import Files", font=ctk.CTkFont(size=22, weight="bold"), anchor="nw")
        self.current_step.grid(row=0, column=0, padx=20, pady=10, sticky="nw")
        # create label with explanation
        self.current_step_ex = ctk.CTkLabel(App.frames[frame_id], fg_color="transparent", text="Import Files explanation", font=ctk.CTkFont(size=12), anchor="nw")
        self.current_step_ex.grid(row=1, column=0, padx=25, pady=10, sticky="nw")
        # action button
        self.action_button = ctk.CTkButton(App.frames[frame_id], text='Select folder', command=self.select_folder)
        self.action_button.grid(row=1, column=2, padx=20, pady=10, sticky="se")

        # Set blue frame's row and column weights to make it take the entire space
        App.frames[frame_id].grid_rowconfigure(1, weight=1)
        App.frames[frame_id].grid_columnconfigure(0, weight=1)

    def working_space(self, parent, frame_id, color):
        # Add a frame on top of the right panel with blue background
        App.frames[frame_id] = ctk.CTkFrame(parent, fg_color=color)
        App.frames[frame_id].grid(row=2, column=0,  padx=5, pady= 5, sticky="nsew")

        # Add widgets to the blue frame
        bt_from_frame2 = ctk.CTkButton(App.frames[frame_id], text="Test 3", command=lambda: print(color))
        bt_from_frame2.grid(row=0, column=0, padx=20, pady=10)

        # Set blue frame's row and column weights to make it take the entire space
        App.frames[frame_id].grid_rowconfigure(0, weight=1)
        App.frames[frame_id].grid_columnconfigure(0, weight=1)

    def navigation_frame(self, parent, frame_id):
        # Add a frame on top of the right panel with blue background
        App.frames[frame_id] = ctk.CTkFrame(parent, fg_color="purple")
        App.frames[frame_id].grid(row=3, column=0, padx=5, pady= 5,  sticky="nsew")

        # Add widgets to the blue frame
        bt_navigation = ctk.CTkButton(App.frames[frame_id], text="Test 4", command =  lambda: self.show_frame("WSFrame1"))
        bt_navigation.grid(row=0, column=0, padx=20, pady=10, sticky="w")

        # Add widgets to the blue frame
        bt_navigation2 = ctk.CTkButton(App.frames[frame_id], text="Test 5", command =  lambda: self.show_frame("WSFrame2"))
        bt_navigation2.grid(row=0, column=2, padx=20, pady=10, sticky="e")

        # Set blue frame's row and column weights to make it take the entire space
        App.frames[frame_id].grid_rowconfigure(0, weight=1)
        App.frames[frame_id].grid_columnconfigure(0, weight=1)


    def __init__(self):
        super().__init__()

        # configure window
        self.title("VisuaLite V0.1")
        self.geometry(f"{1100}x{580}")

        self.grid_columnconfigure(0, weight=0)
        self.grid_columnconfigure(1, weight=1)

        # -------------------------------------------------------------------------------------------------------------- left side panel
        self.left_side_panel = ctk.CTkFrame(self, corner_radius=8, width=300)
        self.left_side_panel.grid(row=0, column=0, rowspan=8, sticky="nsew", padx=(20, 10), pady=20)

        self.left_side_widgets(self.left_side_panel)

        # Set left_side_panel's row weight to make it occupy full height
        self.grid_rowconfigure(0, weight=1)

        # -------------------------------------------------------------------------------------------------------------- right side panel
        self.right_side_panel = ctk.CTkFrame(self, corner_radius=8)
        self.right_side_panel.grid(row=0, column=1, sticky="nsew", padx=(10, 20), pady=20)
        
        # Set right_side_panel's row and column weights to make the blue_frame expand to fill the available space
        self.right_side_panel.grid_rowconfigure(2, weight=1)
        self.right_side_panel.grid_columnconfigure(0, weight=1)

        #Create frames inside right side panel
        App.frames["BCFrame"] = BreadcrumbFrame(self.right_side_panel)
        App.frames["BCFrame"].grid(row=0, column=0, padx=5, pady= 5, sticky="nsew")
        
        self.current_step(self.right_side_panel, "CSFrame")
        self.working_space(self.right_side_panel, "WSFrame1", "green")
        self.working_space(self.right_side_panel, "WSFrame2", "lightgreen")
        self.navigation_frame(self.right_side_panel, "NFrame")

        


    # method to change frames
    def show_frame(self, frame_id):
        if App.current is not None:
            App.frames[App.current].grid_forget() # Hide the current frame

        App.frames[frame_id].grid(row=2, column=0, sticky="nsew") # Show the selected frame
        App.current = frame_id

    def change_appearance_mode_event(self, new_appearance_mode: str):
        ctk.set_appearance_mode(new_appearance_mode)

    def change_scaling_event(self, new_scaling: str):
        new_scaling_float = int(new_scaling.replace("%", "")) / 100
        ctk.set_widget_scaling(new_scaling_float)

    def checkbox_frame_event(self):
        print(f"checkbox frame modified: {self.scrollable_checkbox_frame.get_checked_items()}")

    def select_folder(self):
        print("button Select folder pressed")
        self.dirname = fd.askdirectory(parent=self,initialdir=PATH,title='Please select a directory')
        print(self.dirname)
        

        global csv_files_list
        csv_files_list = []
        for filename in os.listdir(self.dirname):
            if filename.lower().endswith('.csv'):
                csv_files_list.append(filename)

        tk.messagebox.showinfo(title='Information', message=str(len(csv_files_list)) + ' files found in the folder selected: ' + self.dirname)

        App.frames["FilesUpload"] = ScrollableCheckBoxFrame(self.right_side_panel, command=self.checkbox_frame_event,
                                                                 item_list=csv_files_list)
        App.frames["FilesUpload"].grid(row=2, column=0, padx=5, pady=5,  sticky="nsew")
        

a = App()
a.mainloop()
